In [2]:
import pandas as pd
import numpy as np
import os
import re
import gzip
from tqdm import tqdm
from multiprocessing import Pool
import glob
import multiprocessing
import json
import ast

In [7]:
oligo_df = pd.read_excel('../test/data/target_info/LVOTUMIv7_untested_ALT_oligos_sgRNAs.xlsx', sheet_name='OT_oligo_designs_ORIGINAL', skiprows=1).rename(columns={'Unnamed: 1':'Barcode'})
oligo_df['OT_sequence'] = oligo_df['~3 upstream, ~20 protospacer, 3 PAM, 6 downstream'].str.upper()
oligo_df['OT'] = oligo_df['OT'].str.replace('_','-')
oligo_df = oligo_df.set_index('OT')[['Barcode', 'OT_sequence']]


In [8]:
oligo_df.to_csv('../test/data/target_info/target_oligos_sequenes.csv')

In [ ]:
# tar xzvf alleles_HiFi_WTCas9_NoEP.tar.gz

In [6]:
file_path = '/Users/jieconglin/Documents/GitHub/ABSOLVE-seq/test/absolveseq_edits/raw/'

# 获取当前文件的目录
current_dir = os.path.dirname(file_path)
# 获取上一级目录
parent_dir = os.path.dirname(current_dir)
print("Parent Directory:", parent_dir)

Parent Directory: /Users/jieconglin/Documents/GitHub/ABSOLVE-seq/test/absolveseq_edits


In [ ]:
# Functions to process UMI alleles
def filter_low_reads_umiallele(df):
    filtered_df = df[df['#Reads'] > 1].reset_index()
    filtered_df['%Reads'] = filtered_df['#Reads']/filtered_df['#Reads'].sum() * 100
    return filtered_df

def deduplicate_umiallele(df, filtering=False):
    tdf = df.sort_values(by=['UMI', '%Reads_UMI'], ascending=False)
    tdf_dedup = tdf.drop_duplicates(subset='UMI', keep='first')
    if filtering:
        filtered_df = tdf_dedup[tdf_dedup['%Reads_UMI'] > 65].reset_index()
    else:
        filtered_df = tdf_dedup.reset_index()
    filtered_df['#Reads'] = 1
    filtered_df['%Reads'] = filtered_df['#Reads']/filtered_df['#Reads'].sum() * 100
    return filtered_df

def levenshteinFullMatrix(str1, str2):
    m = len(str1)
    n = len(str2)
    # Initialize a matrix to store the edit distances
    dp = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    # Initialize the first row and column with values from 0 to m and 0 to n respectively
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Fill the matrix using dynamic programming to compute edit distances
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                # Characters match, no operation needed
                dp[i][j] = dp[i - 1][j - 1]
            else:
                # Characters don't match, choose minimum cost among insertion, deletion, or substitution
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1])

    # Return the edit distance between the strings
    return dp[m][n]

def add_cutsite(fn):
    out_name = fn.split('/')[-1]
    allele_full_df = pd.read_csv(fn, sep='\t')
    align_20bp = []
    ref_20bp = []
    for idx, row in allele_full_df.iterrows():
        ref_pos = np.array(ast.literal_eval(row['ref_positions']))
        cutsite = row['sgRNA_cut_points']
        cutsite = np.arange(len(row['Reference_Sequence']))[ref_pos == cutsite][0]
        align_20bp.append(row['Aligned_Sequence'][cutsite-9:cutsite+11])
        ref_20bp.append(row['Reference_Sequence'][cutsite-9:cutsite+11])
    allele_full_df['Unedited'] = allele_full_df['Read_Status'] == 'UNMODIFIED'
    allele_full_df['Aligned_Sequence_20bp'] = align_20bp
    allele_full_df['Reference_Sequence_20bp'] = ref_20bp
    allele_full_df.to_csv(fn, sep='\t', index=False)
    return fn

In [ ]:
# Add cutting site to allele table
allele_table_list = []
allele_table_dir = './alleles_HiFi_WTCas9_NoEP/'
for fn in glob.glob(allele_table_dir+'/*_full_withUMI.txt'):
    allele_table_list.append(fn)
with Pool(processes=10) as pool:
    results = []
    for result in tqdm(pool.imap_unordered(add_cutsite, allele_table_list), total=len(allele_table_list)):
        results.append(result)

## Load UMI annotation from plasmid pools

In [ ]:
maxi_ds_df = pd.read_csv('./plasmid_pools_data/trimmed_LV-Maxi-merged3_L001_R1_001.fastq.gz.extractedSeq.UMI-Barcode-dedup-dedup.annot.csv')
maxi_ds_df['good_L0'] = maxi_ds_df['with_expected_OT']
maxi_ds_df['good_S'] = (maxi_ds_df['#UMI-Barcode'] == maxi_ds_df['#UMI-Seq-merged'])&(maxi_ds_df['with_expected_OT'])
umi_complexity = pd.read_csv('./plasmid_pools_data/UMI_entropy_barcode_count_merged_plasmid_pools.csv', index_col='umi_r1')

## Annotate Raw allele based on Barcode annotation

In [ ]:
os.makedirs('./target_umi_barcode_table_WTCas9_NoEP', exist_ok=True)
out_folder = './target_umi_barcode_table_WTCas9_NoEP/raw/'
os.makedirs(out_folder, exist_ok=True)
# allele_table_dir = './alleles_HiFi_WTCas9_NoEP/'
for fn in tqdm(glob.glob(allele_table_dir + '/*_full_withUMI.txt')):
    ot_name = fn.split('/')[-1].split('_')[0]
    if ot_name not in sel_OTs:
        continue
    out_name = fn.split('/')[-1]
    allele_df = pd.read_csv(fn, sep='\t')[['UMI', 'index', 'Aligned_Sequence',
               'Reference_Sequence', 'Aligned_Sequence_20bp', 'Reference_Sequence_20bp', 'sgRNA_cut_points', 'Aligned_Reference_Scores', 'Unedited', 'n_deleted', 'n_inserted',
               'n_mutated', '#Reads', '%Reads_UMI', '%Reads', 'sample_id', 'donor',
               'replicate', 'group', 'OT_name']]
    ot_name = allele_df.iloc[0]['OT_name'].replace('-', '_')
    OT_maxi_df = maxi_ds_df[(maxi_ds_df['OT'] == ot_name)].drop_duplicates(subset='umi_r1', keep='first')
    umi_set = list(set(allele_df['UMI'].unique()).intersection(umi_complexity.index))
    allele_df = allele_df.merge(umi_complexity.loc[umi_set][['#barcode', 'entropy']], left_on='UMI', right_index=True, how='left')
    allele_maxi_df = allele_df.merge(OT_maxi_df, how='left', left_on='UMI', right_on='umi_r1')
    allele_maxi_df.loc[allele_maxi_df[allele_maxi_df['good_L0'].isna()].index, 'good_L0'] = 'Unseen'
    allele_maxi_df.loc[allele_maxi_df[allele_maxi_df['with_expected_OT'].isna()].index, 'with_expected_OT'] = 'Unseen'
    allele_maxi_df['good_L0'] = allele_maxi_df['good_L0'].astype(str)
    # Stringent
    allele_maxi_df.loc[allele_maxi_df[allele_maxi_df['good_S'].isna()].index, 'good_S'] = 'Unseen'
    allele_maxi_df.loc[allele_maxi_df[allele_maxi_df['with_expected_OT'].isna()].index, 'with_expected_OT'] = 'Unseen'
    allele_maxi_df['good_S'] = allele_maxi_df['good_S'].astype(str)
    allele_maxi_df['with_expected_OT'] = allele_maxi_df['with_expected_OT'].astype(str)
    allele_maxi_df.to_csv(out_folder + '/' + out_name.replace('.txt', '.annot.txt'), index=False, sep='\t')

## Filter alleles based on UMI annotetion (Dedud, Dedup and Dedud+dedup)

In [ ]:
dir_path = './target_umi_barcode_table_WTCas9_NoEP/' # output folder
for fn in tqdm(glob.glob(dir_path +'/raw/*.annot.txt')):
    out_name = fn.split('/')[-1]
    allele_df = pd.read_csv(fn, sep='\t')
    # allele_df['UMI-Aligned_Sequence'] = allele_df['UMI'] + '-' + allele_df['Aligned_Sequence']
    donor = allele_df.iloc[0]['donor']
    edit = allele_df.iloc[0]['group']
    allele_deduplicate_df = deduplicate_umiallele(allele_df)# .drop(columns='level_0')
    out_folder = '{}/raw_dedup/'.format(dir_path)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_fn = out_folder + out_name
    allele_deduplicate_df.to_csv(out_fn, sep='\t', index=False)

    for method in ['S']:
        umi_type_name = 'good_' + method
        dedud_df = allele_df[allele_df[umi_type_name] == 'True'].reset_index()
        dedud_df['%Reads'] = dedud_df['#Reads']/dedud_df['#Reads'].sum() * 100
        dedud_df = dedud_df.drop(columns='level_0')
        out_folder = dir_path + '/dedud{}/'.format(method)
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        dedud_df.to_csv(out_folder + out_name, sep='\t', index=False)

        # allele_deduplicate_df = deduplicate_umiallele(dedud_df).drop(columns='level_0')
        # out_folder = './target_umi_barcode_table_full/dedud{}_dedup/'.format(method)
        # if not os.path.exists(out_folder):
        #     os.mkdir(out_folder)
        # out_fn = out_folder + out_name
        # allele_deduplicate_df.to_csv(out_fn, sep='\t')

        dedud_filtered_df = dedud_df[(dedud_df['entropy'] > 1.477)&(dedud_df['#barcode'] == 1)|(dedud_df['#barcode'] == 2)]
        out_folder = dir_path + '/dedud{}_filtered/'.format(method)
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        dedud_filtered_df.to_csv(out_folder + out_name, sep='\t', index=False)

        allele_deduplicate_df = deduplicate_umiallele(dedud_filtered_df).drop(columns='level_0')
        out_folder = dir_path + '/dedud{}_filtered_dedup/'.format(method)
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        out_fn = out_folder + out_name
        allele_deduplicate_df.to_csv(out_fn, sep='\t')

## Filter recombination edited alleles

In [ ]:
# Load expected OT sequences
sample_df = pd.read_excel('./plasmid_pools_data/NovaSeq3_sample_info.xlsx', sheet_name='Sheet3')
amplicon_df = pd.read_csv('./plasmid_pools_data/OT_guide_amplicon_seq.csv', index_col=['OT-Name'])
oligo_df = pd.read_excel('./plasmid_pools_data/LVOTUMIv7_untested_ALT_oligos_sgRNAs.xlsx', sheet_name='OT_oligo_designs_ORIGINAL', skiprows=1).rename(columns={'Unnamed: 1':'Barcode'})
oligo_df['OT_sequence'] = oligo_df['~3 upstream, ~20 protospacer, 3 PAM, 6 downstream'].str.upper()
oligo_df = oligo_df.set_index('OT')[['Barcode', 'OT_sequence']]

In [ ]:
# Annotate recombination edits
for filter in ['dedudS_filtered']:
    print(filter)
    for fn in tqdm(glob.glob(dir_path + '/{}/*.annot.txt'.format(filter))):
        out_name = fn.split('/')[-1]
        ot_name = '_'.join(out_name.split('_')[0:4]).replace('_', '-')
        barcode = oligo_df.loc[ot_name.replace('-', '_'), 'Barcode']
        constant = 'CCAACCTCATAGAACACTCATCC'
        target_seq_pos = len(barcode) + len(constant)
        allele_df = pd.read_csv(fn, sep='\t')
        # allele_df = allele_df.drop(columns=['best_match_strigent'])
        allele_df['best_match'] = ot_name
        allele_df['same_match'] = ''
        allele_df['#same_match'] = 0
        allele_edited = allele_df[allele_df['Unedited'] == False]
        recomb_edits_idx = []
        mis_targets = []
        mis_targets_score = []
        same_edits_idx = []
        same_targets = []
        same_targets_score = []
        n_same_targets = []
        for idx, row in allele_edited.iterrows():
            target_seq = row['Aligned_Sequence'].replace('-', '')
            target_seq = target_seq[target_seq_pos:]
            expected_dist = levenshteinFullMatrix(target_seq, oligo_df.loc[ot_name.replace('-', '_'), 'OT_sequence'])
            current_dist = expected_dist
            # print(ot_name, target_seq, oligo_df.loc[ot_name.replace('-', '_'), 'OT_sequence'], current_dist)
            best_match = ot_name
            same_match = []
            # best_match_S = ot_name
            for ot, row in oligo_df.iterrows():
                ot = ot.replace('_', '-')
                if ot == ot_name:
                    continue
                tdist = levenshteinFullMatrix(target_seq, row['OT_sequence'])
                if tdist < current_dist:
                    # print('\t', ot, tdist, target_seq, row['OT_sequence'])
                    best_match = ot
                    current_dist = tdist
                if tdist == expected_dist:
                    same_match.append(ot)

            if best_match != ot_name:
                recomb_edits_idx.append(idx)
                mis_targets.append(best_match)
                mis_targets_score.append(current_dist)
            if len(same_match) > 0:
                same_edits_idx.append(idx)
                same_targets.append(','.join(same_match))
                n_same_targets.append(len(same_match))
                same_targets_score.append(expected_dist)
                # print(ot_name, best_match)
        allele_df.loc[recomb_edits_idx, 'best_match'] = mis_targets
        allele_df.loc[same_edits_idx, 'same_match'] = same_targets
        allele_df.loc[same_edits_idx, '#same_match'] = n_same_targets
        allele_df.loc[recomb_edits_idx, 'best_match_score'] = mis_targets_score
        allele_df.loc[same_edits_idx, 'same_match_score'] = same_targets_score
        allele_df.to_csv(fn, sep='\t', index=False)

In [ ]:
# Filter recombination edits
for fn in tqdm(glob.glob(dir_path + '/dedudS_filtered/*.annot.txt')):
    out_name = fn.split('/')[-1]
    allele_df = pd.read_csv(fn, sep='\t')
    allele_df = allele_df[(allele_df['OT_name'] == allele_df['best_match'])&(allele_df['#same_match'] == 0)].reset_index(drop=True)
    out_folder = dir_path + '/dedudS_filtered_recomb/'
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_fn = out_folder + out_name
    allele_df.to_csv(out_fn, sep='\t', index=False)
    # allele_df['UMI-Aligned_Sequence'] = allele_df['UMI'] + '-' + allele_df['Aligned_Sequence']
    allele_deduplicate_df = deduplicate_umiallele(allele_df)# .drop(columns='level_0')
    out_folder = dir_path + '/dedudS_filtered_recomb_dedup/'
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_fn = out_folder + out_name
    allele_deduplicate_df.to_csv(out_fn, sep='\t', index=False)

## Edit rate estimation and power analysis